In [1]:
from PIL import Image
from transformers import ViTImageProcessor, ViTModel, CLIPImageProcessor, CLIPModel

/home/lukasz/miniconda3/envs/cudaenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import ViTForImageClassification

In [3]:
vit_image_processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")
clip_image_processor = CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32")

img = Image.open("/home/lukasz/binary-hyper-maml/filelists/emnist/emnist/13/11.png").convert('RGB')

In [4]:
img

In [5]:
img.size

(28, 28)

In [6]:
inputs = vit_image_processor(images=img, return_tensors="pt")
pixel_values = inputs.pixel_values

In [7]:
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

In [8]:
outptut = model(pixel_values)

In [9]:
outptut[0].shape

torch.Size([1, 1000])

In [10]:
feature_extractor = ViTModel.from_pretrained('google/vit-base-patch16-224')

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
outputs = feature_extractor(pixel_values)
sequence_output = outputs[0]

sequence_output[:, 0, :].shape

torch.Size([1, 768])